In [3]:
import numpy as np
import pygad
import random
from hybrid_solution_data_loader import get_data
import copy
from models import Workstation, Task, Recipe, Order, SimulationEnvironment, Resource
from visualize import visualize_schedule

In [1]:
def translate(n_workstations, data_recipes, operation_times, last_slot = 5000):
    tasks = []
    recipes = []
    orders = []
    workstations = []
    resources = []
    start = 0
    for i in range(len(operation_times)):
        tasks.append(Task(i, f't{i}', [], [], [], [], True, 0, 0))
    for i in range(n_workstations):
        task_durations = []
        for j in range(len(operation_times)):
            task_durations.append((tasks[j].id, operation_times[j][i])) # NOTE: usually id is used instead of task object
        workstations.append(Workstation(i, f'w{i}', [], task_durations))
    r_id = 0
    for recipe in data_recipes:
        # NOTE: every "recipe" in this dataset is the amount of rows in operation_times which belong to the "recipe"
        tasks_for_recipe = []
        for i in range(start, start + recipe):
            tasks_for_recipe.append(tasks[j])
        recipes.append(Recipe(r_id, f'r{r_id}', tasks_for_recipe))
        resources.append(Resource(r_id, f'recipe_r{r_id}_placeholder_resource', 0, 0, False, [r_id], 0))
        start += recipe
        r_id += 1
    for i in range(len(resources)):
        orders.append(Order(i, 0, last_slot, last_slot, resources[i], 0, 0, False, 0, True)) # TODO: insert last slot as delivery date, NOTE 2: instead of ordered resources, recipes are inserted as ordered resources for this example
    simulation_environment = SimulationEnvironment(workstations, tasks, resources, recipes)
    return simulation_environment, orders

def to_input(operation_times, recipes):
    input_format = [0 for _ in range(len(operation_times) * 2)]
    orders = [i for i in range(len(recipes))] # not really needed
    return input_format, orders

In [5]:
n_workstations, recipes, operation_times = get_data(0) # note: every task can be processed on all workstations in this dataset
worst_case = 0
for operation in operation_times:
    max = -float('inf')
    for duration in operation:
        if duration > max:
            max = duration
    worst_case += max
print(f'Worst possible schedule duration: {worst_case}')
last_slot = worst_case
simulation_environment, orders = translate(n_workstations, recipes, operation_times)
input_format, orders = to_input(operation_times, recipes) # orders?

Worst possible schedule duration: 130


In [5]:
def similarity(a, b):
    # used encoding: <assignments, sequence> 
    distance = 0
    pivot = int(len(a)/2)
    for i in range(0, pivot):
        if a[i] != b[i]: # workstation assignment
            distance += len(n_workstations) # get amount of alternative workstations for the respective job
        if a[pivot+i] != b[pivot+i]: # job sequence
            distance += 1 
    return distance

def validate_sequence(individual, order_job_amounts):
    counted = copy.deepcopy(order_job_amounts)
    for i in range(len(individual)/2, len(individual)):
        counted[individual[i]] -= 1
    return all(x == 0 for x in counted)

def validate_assignments(individual):
    return all(individual[i] < n_workstations and individual[i] >= 0 for i in range(len(individual)/2))

def is_first(jobs, index):
    pass

def construct_schedule(individual):
    # for each job:
    # check if job is first for it's order
    # start_o = 0
    # else
    # start_o = end of previous job for order
    # check if job is first on workstation
    # start_w = 0
    # else
    # start_w = end of previous job on workstation
    # set start point as max(start_o, start_w)
    pass

def crossover_function(parents, offspring_size, ga_instance):
    # uniform crossover for assignments
    parent1 = parents[0].copy()
    parent2 = parents[1].copy()
    offspring_assignemnts = []
    for i in range(len(operation_times)): # assumption that every recipe is ordered exactly once
        if random.random() < 0.5:
            offspring_assignemnts.append(parent1[i])
        else:
            offspring_assignemnts.append(parent2[i])
    offspring_sequence = []
    # iPOX crossover for sequence
    for i in range(len(operation_times), 2 * len(operation_times)): # assumption that every recipe is ordered exactly once
        pass
    # -> select parent
    # -> randomly generate 2 job subsets
    # -> copy genes belonging to the jobs in the subsets
    # -> fill in the gaps in order with the rest
    offspring = np.concatenate([offspring_assignemnts, offspring_sequence]) # concats 2 python arrays, returns np array
    return offspring

def selection_function(fitness, num_parents, ga_instance):
    fitness_sorted = sorted(range(len(fitness)), key=lambda k: fitness[k])
    fitness_sorted.reverse()

    parents = np.empty((num_parents, ga_instance.population.shape[1]))
    # for each individual
    # FN = sum ( 1 / (c_max(individual) * neighbourhood) )
    # selection_probability_k = ( 1 / (c_max(individual) * neighbourhood) ) / FN # selection probability for each individual
    return parents, fitness_sorted[:num_parents] # replace with actual fitness of chosen parents

def c_max(individual):
    for i in range(len(individual)/2, len(individual)): # find first and last assignment for each workstation
        pass
    pass

def fitness_function(solution, solution_idx):
    fitness = 0
    if validate_sequence(solution, None) and validate_assignments(solution):
        fitness = 1 / c_max(solution)
        pass
    else:
        fitness = 0
        pass
    return fitness

def mutation_function(offsprings, ga_instance):
    pass

def stage_1(): # GA for feasible solutions
    num_generations = 1000
    num_parents_mating = 50
    sol_per_pop = 100
    init_range_low = 0
    init_range_high = len(orders) # doesn't really matter
    keep_parents = 10
    mutation_percent_genes = 10 # doesn't really matter

    crossover_type = crossover_function
    mutation_type = mutation_function
    fitness_func = fitness_function
    parent_selection_type = selection_function

    num_genes = len(input_format) * 2
    gene_type = int
    gene_space = []

    for i in range(int(num_genes / 2)):
        # set space for the first half (assignments)
        gene_space.append(n_workstations)
    for i in range(int(num_genes / 2), num_genes):
        # set space for the second half (sequence)
        gene_space.append(len(orders))

    ga_instance = pygad.GA(num_generations=num_generations, num_parents_mating=num_parents_mating, fitness_func=fitness_func, sol_per_pop=sol_per_pop, num_genes=num_genes, init_range_low=init_range_low, init_range_high=init_range_high, parent_selection_type=parent_selection_type, keep_parents=keep_parents, crossover_type=crossover_type, mutation_type=mutation_type, mutation_percent_genes=mutation_percent_genes, gene_type=gene_type, gene_space=gene_space)
    ga_instance.run()
    solution, solution_fitness, solution_idx = ga_instance.best_solution()
    print("Parameters of the best solution : {solution}".format(solution=solution))
    print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=abs(solution_fitness) - 1))
    return ga_instance

In [6]:
n_clusters = 3

def stage_2(population): # clustering of GA result population
    clusters = []
    for i in range(n_clusters):
        clusters.append([]) # create clusters using indices
    for i in range(len(population)):
        pass
    pass

In [7]:
def get_neighbours(individual):
    # change position of an operation to alternative machine
    # create new schedule sequence accordingly, create n solutions
    pass

def calculate_fitness(individual):
    # rebuild schedule
    schedule = construct_schedule(individual)
    # makespan only
    fitness = c_max(individual)
    return fitness

def move_and_insert(individual):
    pass

def stage_3(clusters): # tabu search for best solution
    overall_best = None
    overall_best_fitness = None
    for individuals in clusters:
        best = individuals[0]
        if overall_best is None:
            overall_best = best
            overall_best_fitness = calculate_fitness(overall_best)
        max_tabu_size = int(len(individuals) / 4) # for testing
        best_candidate = individuals[0]
        best_fitness = best_candidate_fitness = calculate_fitness(best_candidate)
        tabu_list = []
        tabu_list.append(individuals[0])
        stop = False
        max_iter = 1000
        i = 0
        while not stop:
            neighbours = get_neighbours(best_candidate)
            best_candidate = neighbours[0]
            for candidate in neighbours:
                if candidate not in tabu_list:
                    candidate_fitness = calculate_fitness(candidate)
                    if candidate_fitness < best_candidate_fitness:
                        best_candidate = candidate
                        best_candidate_fitness = candidate_fitness
                        if candidate_fitness < overall_best_fitness:
                            overall_best = candidate
                            overall_best_fitness = candidate_fitness
            if best_candidate_fitness < best_fitness:
                best = best_candidate
                best_fitness = best_candidate_fitness
            tabu_list.append(best_candidate)
            if len(tabu_list) > max_tabu_size:
                tabu_list.pop()
            stop = i >= max_iter
            i+=1
    return overall_best

In [8]:
end_population = stage_1()
clusters = stage_2(end_population)
solution = stage_3(clusters)
schedule = construct_schedule(solution)
visualize_schedule(schedule, simulation_environment)